### Import libraries and create DataFrame from csv 

In [262]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("WR_Data_2025.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Ja'Marr Chase (CIN),127,175,1708,13.4,70,19,17,3,32,0,17,403.0,23.7
1,2,Justin Jefferson (MIN),103,154,1533,14.9,97,28,10,1,3,0,17,317.5,18.7
2,3,Amon-Ra St. Brown (DET),115,141,1263,11.0,66,14,12,2,6,0,17,316.2,18.6
3,4,Brian Thomas Jr. (JAC),87,133,1282,14.7,85,18,10,6,48,0,17,284.0,16.7
4,5,Drake London (ATL),100,158,1271,12.7,39,12,9,1,-3,0,17,280.8,16.5
5,6,Malik Nabers (NYG),109,170,1204,11.0,59,16,7,5,2,0,15,273.6,18.2
6,7,Terry McLaurin (WAS),82,117,1096,13.4,86,12,13,2,2,0,17,267.8,15.8
7,8,CeeDee Lamb (DAL),101,152,1194,11.8,65,16,6,14,70,0,15,263.4,17.6
8,9,Jaxon Smith-Njigba (SEA),100,137,1130,11.3,46,14,6,5,26,0,17,253.0,14.9
9,10,Garrett Wilson (NYJ),101,154,1104,10.9,42,16,7,2,5,0,17,251.9,14.8


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [263]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/R', 'LG', 'G', 'FPTS', 'FPTS/G']

# Base columns that will get a /game version
base_cols = [c for c in per_game if c not in exclude_per_game]
per_game_cols = [f'{c}/game' for c in base_cols]

# Create per-game columns (safe divide if G can be 0)
for col in base_cols:
    df[f'{col}/game'] = df[col].div(df['G'].replace(0, np.nan)).round(1).fillna(0)

# Keep using THIS for downstream analysis (unchanged behavior)
final_columns = exclude_per_game + per_game_cols

# Separate list just for showing original next to per-game
display_cols = []
for col in per_game:
    if col in exclude_per_game:
        if col in df.columns:
            display_cols.append(col)                 # keep excluded as-is
    else:
        display_cols.extend([col, f'{col}/game'])    # interleave original + per-game

# Display: originals next to per-game
df[['Rank', 'Player'] + display_cols].head(10)


# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
#for col in per_game:
#    if col not in exclude_per_game:
#        df[col + '/game'] = (df[col] / df['G']).round(1)
#
# Define final columns to be used for analysis, combining excluded and new per-game columns
#final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]
#
# Display final columns with 'Rank' and 'Player'
#df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,REC,REC/game,TGT,TGT/game,YDS,YDS/game,Y/R,LG,...,TD/game,ATT,ATT/game,YDS.1,YDS.1/game,TD.1,TD.1/game,G,FPTS,FPTS/G
0,1,Ja'Marr Chase (CIN),127,7.5,175,10.3,1708,100.5,13.4,70,...,1.0,3,0.2,32,1.9,0,0.0,17,403.0,23.7
1,2,Justin Jefferson (MIN),103,6.1,154,9.1,1533,90.2,14.9,97,...,0.6,1,0.1,3,0.2,0,0.0,17,317.5,18.7
2,3,Amon-Ra St. Brown (DET),115,6.8,141,8.3,1263,74.3,11.0,66,...,0.7,2,0.1,6,0.4,0,0.0,17,316.2,18.6
3,4,Brian Thomas Jr. (JAC),87,5.1,133,7.8,1282,75.4,14.7,85,...,0.6,6,0.4,48,2.8,0,0.0,17,284.0,16.7
4,5,Drake London (ATL),100,5.9,158,9.3,1271,74.8,12.7,39,...,0.5,1,0.1,-3,-0.2,0,0.0,17,280.8,16.5
5,6,Malik Nabers (NYG),109,7.3,170,11.3,1204,80.3,11.0,59,...,0.5,5,0.3,2,0.1,0,0.0,15,273.6,18.2
6,7,Terry McLaurin (WAS),82,4.8,117,6.9,1096,64.5,13.4,86,...,0.8,2,0.1,2,0.1,0,0.0,17,267.8,15.8
7,8,CeeDee Lamb (DAL),101,6.7,152,10.1,1194,79.6,11.8,65,...,0.4,14,0.9,70,4.7,0,0.0,15,263.4,17.6
8,9,Jaxon Smith-Njigba (SEA),100,5.9,137,8.1,1130,66.5,11.3,46,...,0.4,5,0.3,26,1.5,0,0.0,17,253.0,14.9
9,10,Garrett Wilson (NYJ),101,5.9,154,9.1,1104,64.9,10.9,42,...,0.4,2,0.1,5,0.3,0,0.0,17,251.9,14.8


### Calculate the correlations for the final stats across different conditions

In [264]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G', 'LG']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/R,0.60,0.18,0.00,0.01,0.20
REC/game,0.98,0.96,0.87,0.77,0.89
TGT/game,0.97,0.95,0.82,0.65,0.85
YDS/game,0.99,0.98,0.94,0.89,0.95
20+/game,0.93,0.89,0.59,0.40,0.70
TD/game,0.89,0.85,0.74,0.76,0.81
ATT/game,0.35,0.22,-0.12,-0.21,0.06
YDS.1/game,0.35,0.22,-0.13,-0.23,0.05
TD.1/game,0.24,0.18,-0.11,-0.24,0.02


### Assign the weights for the final stats

In [265]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/R,0.20,0.04,0.07
REC/game,0.89,0.80,0.17
TGT/game,0.85,0.72,0.16
YDS/game,0.95,0.90,0.18
20+/game,0.70,0.49,0.10
TD/game,0.81,0.66,0.15
ATT/game,0.06,0.00,0.06
YDS.1/game,0.05,0.00,0.06
TD.1/game,0.02,0.00,0.06


### Multiply the assigned weights to the final stats and calculate the score

In [266]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,REC/game_weighted,TGT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted
0,1,Ja'Marr Chase (CIN),23.7,1.2,1.6,18.0,0.1,0.1
1,2,Justin Jefferson (MIN),18.7,1.0,1.4,16.1,0.2,0.1
2,3,Amon-Ra St. Brown (DET),18.6,1.1,1.3,13.3,0.1,0.1
3,4,Brian Thomas Jr. (JAC),16.7,0.8,1.2,13.5,0.1,0.1
4,5,Drake London (ATL),16.5,1.0,1.4,13.4,0.1,0.1
5,6,Malik Nabers (NYG),18.2,1.2,1.8,14.4,0.1,0.1
6,7,Terry McLaurin (WAS),15.8,0.8,1.1,11.5,0.1,0.1
7,8,CeeDee Lamb (DAL),17.6,1.1,1.6,14.2,0.1,0.1
8,9,Jaxon Smith-Njigba (SEA),14.9,1.0,1.3,11.9,0.1,0.1
9,10,Garrett Wilson (NYJ),14.8,1.0,1.4,11.6,0.1,0.1


### Model training

In [267]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.034748057124262356
Test Set MSE: 0.022874202002096125
Random Forest Test Set MSE: 0.04979683675090105


/tmp/ipykernel_1708/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [268]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT', 'YDS.1', 'TD.1', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# # Exclude the specified columns from final_columns
# final_columns = [col for col in final_columns if col not in final_columns_exclude]

# # Create final analysis columns
# analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
# analysis.set_index('Rank', inplace=True)
# analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Build export columns by filtering the interleaved list you created earlier
# (display_cols already has [original, original/game] pairs in the right order)
export_cols = [c for c in display_cols if c not in set(final_columns_exclude) and c in df.columns]

# If you want to pin certain stats at the top in pairs, do it here (optional):
pinned = ['G', 'FPTS', 'FPTS/G']
export_cols = pinned + [c for c in export_cols if c not in pinned]
# pinned_pairs = [x for p in pinned for x in (p, f'{p}/game') if x in export_cols]
# export_cols = pinned_pairs + [c for c in export_cols if c not in pinned_pairs]

# Build the final analysis view using the interleaved columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + export_cols] \
            .set_index('Rank') \
            .sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("WR_Analysis.xlsx", index=True)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,G,FPTS,FPTS/G,REC,REC/game,TGT,TGT/game,YDS,YDS/game,20+,20+/game,TD,TD/game
Rank,,,,,,,,,,,,,,,,,
1,1,Ja'Marr Chase (CIN),10.00,0,17,403.0,23.7,127,7.5,175,10.3,1708,100.5,19,1.1,17,1.0
53,2,Chris Godwin (TB),9.45,51,7,137.8,19.7,50,7.1,62,8.9,576,82.3,7,1.0,5,0.7
26,3,Puka Nacua (LAR),9.44,23,11,206.6,18.8,79,7.2,106,9.6,990,90.0,11,1.0,3,0.3
2,4,Justin Jefferson (MIN),9.33,-2,17,317.5,18.7,103,6.1,154,9.1,1533,90.2,28,1.6,10,0.6
6,5,Malik Nabers (NYG),8.97,1,15,273.6,18.2,109,7.3,170,11.3,1204,80.3,16,1.1,7,0.5
23,6,Nico Collins (HOU),8.95,17,12,210.6,17.6,68,5.7,99,8.2,1006,83.8,14,1.2,7,0.6
20,7,A.J. Brown (PHI),8.86,13,13,216.9,16.7,67,5.2,97,7.5,1079,83.0,17,1.3,7,0.5
17,8,Tee Higgins (CIN),8.85,9,12,222.1,18.5,73,6.1,109,9.1,911,75.9,12,1.0,10,0.8
3,9,Amon-Ra St. Brown (DET),8.84,-6,17,316.2,18.6,115,6.8,141,8.3,1263,74.3,14,0.8,12,0.7
